# Preparing our image for contour generation

### Steps
1. Smoothing
2. Foreground/background segmentation
3. Hole-filling
4. Island removal

### Setup

In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from rotations_helpers import disp_slice
from filters_helpers import get_center_axial_slice

img: sitk.Image = get_center_axial_slice('../ExampleData/MicroBiome_1month_T1w.nii.g')

### Smoothing
First, we smooth the image to reduce noise. We will use the `GradientAnisotropicDiffusionImageFilter` to accomplish this task because of its [edge-preserving properties](https://en.wikipedia.org/wiki/Edge-preserving_smoothing).

In [ ]:
anisotropic_diffusion_filter = sitk.GradientAnisotropicDiffusionImageFilter()

The `GradientAnisotropicDiffusionImageFilter` has [several parameters](https://slicer.readthedocs.io/en/latest/user_guide/modules/gradientanisotropicdiffusion.html) that the user can set. We will set the time step to 0.0625 for a stable solution. 

In [ ]:
STABLE_TIME_STEP = 0.0625

anisotropic_diffusion_filter.SetTimeStep(STABLE_TIME_STEP)

Also, we will set the conductance paramter. The lower its value, the more strongly edges are preserved.

In [ ]:
CONDUCTANCE_PARAMETER = 3.0

smooth_img: sitk.Image = anisotropic_diffusion_filter.SetConductanceParameter(CONDUCTANCE_PARAMETER)
disp_slice(smooth_img)

### Foreground/Background Segmentation
Now we will segment our image into a foreground and background. This helps us eventually generate a contour of our image.

The `OtsuThresholdImageFilter` is one way to segment the image. It automatically computes a threshold.

In [ ]:
otsu_img: sitk.Image = sitk.OtsuThresholdImageFilter().Execute(smooth_img)
disp_slice(otsu_img)

We could also use the `BinaryThresholdImageFilter` if we want to specify a lower and upper threshold.

In [ ]:
LOWER_THRESHOLD = 0.0
UPPER_THRESHOLD = 1.0

binary_threshold_filter: sitk.Image = sitk.BinaryThresholdImageFilter()
binary_threshold_filter.SetLowerThreshold(LOWER_THRESHOLD)
binary_threshold_filter.SetUpperThreshold(UPPER_THRESHOLD)
binary_img: sitk.Image = binary_threshold_filter.Execute(smooth_img)
disp_slice(binary_img)

### Hole-Filling
Often, the result of our foreground/background segmentation has holes. If we were to generate a contour at this point, each of the holes would have their own contour. However, we only want to generate a contour for the head. So, we will use the `BinaryGrindPeakImageFilter` to remove any holes.

**Note:**

The previous processing steps result in an inverted image (i.e. the pixels representing the head are 0 as opposed to 1). For the `BinaryGrindPeakImageFilter` to work as expected, our image must remain inverted.

In [ ]:
no_holes_img: sitk.Image = sitk.BinaryGrindPeakImageFilter().Execute(otsu_img)
disp_slice(no_holes_img)

### Island Removal
Sometimes an image is very noisy. This can result in small islands being scattered throughout the image. These islands will result in the generation of additonal contours, so we will remove them from our image. We can do so by selecting the largest component using the `ConnectedComponentImageFilter`.

**Note:**

The largest component selection algorithm relies upon our image not being inverted. We must use the `NotImageFilter` for a non-inverted image.

In [ ]:
noninverted_img: sitk.Image = sitk.NotImageFilter().Execute(no_holes_img)
component_image = sitk.ConnectedComponentImageFilter().Execute(noninverted_img)
sorted_component_image = sitk.RelabelComponent(
    component_image, sortByObjectSize=True
)
largest_component = sorted_component_image == 1
disp_slice(largest_component)